In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'project': 'qalf',
    'domain': 'tacred',
    'relation': 'per_title',
    'supervision': 'majority',
#     'learn_deps': False,
#     'gen_model_search_space': 1,
#     'gen_init_params': {
#         'lf_propensity'         : True,
#         'class_prior'           : True,
#         'lf_class_propensity'   : True,
#         'seed'                  : None,
#     },
#     'gen_params_default': {
#         'step_size'     : 0.0001,
#         'decay'         : 0.90,
#         'reg_param'     : 0.50,
#         'epochs'        : 25,
#     },    
    'disc_model_class': 'logreg',    
    'disc_model_search_space': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'qalf_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///qalf_tacred.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'], config['project'])
pipe = pipeline(session, candidate_class, config)

Overwriting domain=None to domain=tacred
Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting supervision=generative to supervision=majority
Overwriting project=babble to project=qalf
Using TacredQalfPipeline object.


In [5]:
%time pipe.parse()

Clearing existing...

Running UDF...
[========================================] 100%


Running UDF...
[========================================] 100%


Running UDF...
[========================================] 100%

Parsed 8323 Documents
Parsed 8323 Sentences
CPU times: user 35 s, sys: 3.16 s, total: 38.1 s
Wall time: 37.4 s


In [6]:
%time pipe.extract()

Clearing existing...

Running UDF...
[========================================] 100%

Split 0: Extracted 5201 Candidates

Running UDF...
[========================================] 100%

Split 1: Extracted 1935 Candidates

Running UDF...
[========================================] 100%

Split 2: Extracted 1188 Candidates
CPU times: user 23.1 s, sys: 427 ms, total: 23.5 s
Wall time: 23.3 s


In [ ]:
# %time pipe.load_gold()

In [ ]:
# %time pipe.featurize()

In [ ]:
%time pipe.collect()

In [ ]:
%time pipe.label()

In [ ]:
from snorkel.annotations import load_gold_labels
# L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)
L_gold_dev   = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test  = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
# L_train.lf_stats(session, labels=L_gold_train)
pipe.L_dev.lf_stats(session, labels=L_gold_dev)
# L_test.lf_stats(session, labels=L_gold_test)

In [ ]:
# %time pipe.supervise()

In [ ]:
# %time pipe.classify()